# Time Series Analysis

## Miembros
- Andrés Montoya 21552
- Fernanda Esquivel 21542
- Francisco Castillo 21562
- Fabián Juárez 21440
- Diego Lemus 21469

## Link al repositorio
El repositorio puede ser visualizado [acá](https://github.com/FerEsq/DL-TimeSeries-Analysis)

# Task 1

## Limpieza de datos

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
#Cargar los datos
train_data = pd.read_csv('data/train.csv')

#Convertir la columna 'date' a formato datetime
train_data['date'] = pd.to_datetime(train_data['date'])

#Extraer características de la fecha
train_data['year'] = train_data['date'].dt.year
train_data['month'] = train_data['date'].dt.month
train_data['day_of_week'] = train_data['date'].dt.dayofweek

#Escalar los datos de ventas ('sales') usando StandardScaler
scaler = StandardScaler()
train_data['sales_scaled'] = scaler.fit_transform(train_data[['sales']])

#Verificar los primeros datos transformados
train_data.head()


,date,store,item,sales,year,month,day_of_week,sales_scaled
0,2013-01-01,1,1,13,2013,1,1,-1.362804
1,2013-01-02,1,1,11,2013,1,2,-1.432246
2,2013-01-03,1,1,14,2013,1,3,-1.328083
3,2013-01-04,1,1,13,2013,1,4,-1.362804
4,2013-01-05,1,1,10,2013,1,5,-1.466966


## Preprocesamiento de datos

In [4]:
#Dividir el conjunto de datos en entrenamiento, validación y prueba
train_size = 0.70  # 70% para entrenamiento
validation_size = 0.15  # 15% para validación
test_size = 0.15  # 15% para prueba

total_rows = len(train_data)
train_index = int(total_rows * train_size)
validation_index = int(total_rows * (train_size + validation_size))

train_set = train_data.iloc[:train_index]
validation_set = train_data.iloc[train_index:validation_index]
test_set = train_data.iloc[validation_index:]

In [5]:
#Función para generar secuencias de datos históricos y pronósticos
def create_sequences(data, window_size, forecast_horizon):
    sequences = []
    targets = []
    
    for i in range(len(data) - window_size - forecast_horizon + 1):
        window = data.iloc[i:i + window_size]['sales'].values
        target = data.iloc[i + window_size:i + window_size + forecast_horizon]['sales'].sum()
        sequences.append(window)
        targets.append(target)
    
    return np.array(sequences), np.array(targets)

In [6]:
#Parámetros de secuencias
window_size = 90  #Ventana histórica de 90 días
forecast_horizon = 90  #Pronóstico de 3 meses (90 días)

#Muestra para entrenar y generar secuencias
train_sample = train_set.sample(n=50000, random_state=42)

In [7]:
#Generar secuencias para el conjunto de entrenamiento
train_sequences, train_targets = create_sequences(train_sample, window_size, forecast_horizon)

In [8]:
#Imprimir las dimensiones de las secuencias generadas
print("Dimensiones de las secuencias de entrenamiento:", train_sequences.shape)
print("Dimensiones de los objetivos de entrenamiento:", train_targets.shape)

Dimensiones de las secuencias de entrenamiento: (49821, 90)
Dimensiones de los objetivos de entrenamiento: (49821,)


# Task 2

#### 1. ¿Cuál es el problema del gradiente de fuga en las redes LSTM y cómo afecta la efectividad de LSTM para el pronóstico de series temporales?
* lorem ipsum


#### 2. ¿Cómo se aborda la estacionalidad en los datos de series temporales cuando se utilizan LSTM para realizar pronósticos y qué papel juega la diferenciación en el proceso?
* lorem ipsum

#### 3. ¿Cuál es el concepto de "tamaño de ventana" en el pronóstico de series temporales con LSTM y cómo afecta la elección del tamaño de ventana a la capacidad del modelo para capturar patrones a corto y largo plazo?
* lorem ipsum

### Referencias
1. 